# VacationPy
----Observation:
I noticed when the temperatures across the region are high it also have high humidity with the cloudiness being high. 
I'm assuming there is little to no sunshine with so much clouds no dry heat. being surrounded by a body of water the heat and humidity is very present. 
From the params I tried to find hotels with the criteria of less than 10 mph wind speed, max temp of 80 degrees but no lower than 70 degrees and zero cloudiness only produced less than 2% of the cities met this criteria. It's not to many choices to choose from when trying to find the perfect weather and hotel to stay at from the selected cities. The search would probably have to be widen to get more cities that may be qualified for staying somewhere with perfect weather. 


#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [15]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [28]:
# export csv file from WeatherPY to create into a DataFrame
city_data_csv = "output_data.csv"
city_info = pd.read_csv(city_data_csv)

# Have to convert the Max Temp from Celcius to Fahrenheit 
cols = ['Max Temp']
city_info[cols] = ((city_info[cols] + 32) * 9/5).round(2)
city_info.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Bolshaya Martynovka,47.2710,41.6679,61.22,99,100,4.18,RU,1637688917
1,Dikson,73.5069,80.5464,24.35,97,100,6.70,RU,1637688917
2,Kirakira,-10.4544,161.9205,105.25,82,99,3.86,SB,1637688918
3,Tiksi,71.6872,128.8694,4.63,98,83,3.01,RU,1637688647
4,Sri Aman,1.2376,111.4621,102.69,94,100,0.00,MY,1637688918


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [17]:
# configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = city_info[["Lat", "Lng"]]


# convert humidity as float to use as weight
humidity = city_info["Humidity"].astype(float)



In [18]:
# Plot Heatmap

fig = gmaps.figure(center=(46.0, -5.0), zoom_level=8)
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)



# Add layer
fig.add_layer(heat_layer)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Plot as a Terrain map
fig = gmaps.figure(map_type="TERRAIN")

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [32]:
# weather with 0 cloudiness
no_clouds = city_info[city_info["Cloudiness"] == 0]

# weather with max temp above 70 but lower than 80 degrees
temp_data = no_clouds[(no_clouds["Max Temp"] > 70) & (no_clouds["Max Temp"] < 80)]

# weather with wind speed less 10 mph
less_wind = temp_data[temp_data["Wind Speed"] < 10]

# Drop any null values in all rows
less_wind = less_wind.dropna()
less_wind.head()




,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
66,Busselton,-33.6500,115.3333,79.69,82,0,4.06,AU,1637688942
468,Ardabil,38.2498,48.2933,71.24,60,0,4.44,IR,1637689118


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [38]:
# Store into variable named hotel_df:
hotel_df = less_wind

# Add a "Hotel Name" column to the DataFrame: (setting new columns to hold values)
hotel_df["Hotel Name"] = ""


# Set parameters to search for hotels with 5000 meters:

# Set params:
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Loop through to identify hotels:
for index, row in hotel_df.iterrows():
    
      
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    hotel_data = requests.get(base_url, params=params).json()
    
    try:
        
        hotel_df.loc[index, "Hotel Name"] = hotel_data["results"][0]["name"]
        
    except (KeyError, IndexError):
        print("Hotel not Found, skip")




# Checking hotels:
len(hotel_df[["Lat", "Lng"]])
hotel_df


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
66,Busselton,-33.6500,115.3333,79.69,82,0,4.06,AU,1637688942,Observatory Guest House
468,Ardabil,38.2498,48.2933,71.24,60,0,4.44,IR,1637689118,Hostel ideal


In [23]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]
locations

,Lat,Lng
66,-33.6500,115.3333
468,38.2498,48.2933


In [10]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content= hotel_info)

fig.add_layer(marker_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))